# 인공신경망
실제 인간의 뇌에서 수행되는 신경망 방식을 사용한 컴퓨터 연산 방법   
실제 뇌에서 뉴런간 정보 전달 방식을 모델링한 것

## 퍼셉트론
신경망의 한 종류   

입력값 X 를 가중치와 곱해서 모두 더한 값이 임계값 기준이 있는 노드

In [2]:
import pandas as pd
data = pd.read_csv('data.csv')

In [3]:
data

,x1,x2,y
0,True,True,False
1,True,False,True
2,True,False,True
3,True,True,False
4,True,True,False
...,...,...,...
995,True,True,False
996,True,True,False
997,False,True,True
998,False,False,False


In [4]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x1 = df.iloc[:, 0].values
    self.x2 = df.iloc[:,1].values
    self.y = df.iloc[:,2].values
    self.length = len(df)

  def __getitem__(self, index):
    x = torch.FloatTensor([self.x1[index], self.x2[index]])
    y = torch.FloatTensor([self.y[index]])
    return x,y

  def __len__(self):
    return self.length


class CustomModel(nn.Module):
  def __init__(self):
    super(CustomModel, self).__init__()

    self.layer = nn.Sequential(
        nn.Linear(2,1),     # 2개의 input, 1개의 output 뉴런생성
        nn.Sigmoid()
    )

  def forward(self, x):
    x = self.layer(x)
    return x

train_dataset = CustomDataset('data.csv')
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(10000):
  cost = 0.0
  for x,y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost += loss

  cost = cost/len(train_dataloader)

  if (epoch + 1) % 1000 ==0:
    print(f'Epoch : {epoch+1:4d}, Cost : {cost:.3f}')

with torch.no_grad():
  model.eval()
  input = torch.FloatTensor([
      [0,0],
      [0,1],
      [1,0],
      [1,1]
  ]).to(device)

  outputs = model(input)

print(outputs)

cuda


<ipython-input-4-788e60bf0c93>:16: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
<ipython-input-4-788e60bf0c93>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.692
Epoch : 2000, Cost : 0.692
Epoch : 3000, Cost : 0.691
Epoch : 4000, Cost : 0.692
Epoch : 5000, Cost : 0.692
Epoch : 6000, Cost : 0.692
Epoch : 7000, Cost : 0.692
Epoch : 8000, Cost : 0.692
Epoch : 9000, Cost : 0.692
Epoch : 10000, Cost : 0.692
tensor([[0.4680],
        [0.5008],
        [0.5027],
        [0.5355]], device='cuda:0')


원래 cost(loss)가 줄어들어야 학습이 잘 된다
linear 하나만 사용했기 때문
즉 퍼셉트론 한개로는 문제해결이 어렵다

In [6]:
class fruit():
  def __init__(self) -> None:
    # pass
    self.cnt = 0
    self.name = []

  def add_fruit(self, name, size):
     self.name.append(name)
     self.cnt += 1
     return True

  def __len__(self):
    return self.cnt

In [8]:
fruit_class = fruit()

In [9]:
fruit_class.add_fruit('apple',10)

True

In [10]:
fruit_class.add_fruit('apple',10)
fruit_class.add_fruit('banana',10)
fruit_class.add_fruit('orange',10)

True

In [11]:
len(fruit_class)

4

In [12]:
dir(fruit_class)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_fruit',
 'cnt',
 'name']

In [13]:
fruit_class.get_item()

AttributeError: ignored

In [14]:
fruit_class.name = ['1','2','3']

In [15]:
fruit_class.name

['1', '2', '3']

### 다층퍼셉트론 모델

In [16]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x1 = df.iloc[:, 0].values
    self.x2 = df.iloc[:,1].values
    self.y = df.iloc[:,2].values
    self.length = len(df)

  def __getitem__(self, index):
    x = torch.FloatTensor([self.x1[index], self.x2[index]])
    y = torch.FloatTensor([self.y[index]])
    return x,y

  def __len__(self):
    return self.length


class CustomModel(nn.Module):
  def __init__(self):
    super(CustomModel, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Linear(2,2),
        nn.Sigmoid()
    )
    self.layer2 = nn.Sequential(
        nn.Linear(2,1),
        nn.Sigmoid()
    )

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    return x

train_dataset = CustomDataset('data.csv')
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(10000):
  cost = 0.0
  for x,y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost += loss

  cost = cost/len(train_dataloader)

  if (epoch + 1) % 1000 ==0:
    print(f'Epoch : {epoch+1:4d}, Cost : {cost:.3f}')

with torch.no_grad():
  model.eval()
  input = torch.FloatTensor([
      [0,0],
      [0,1],
      [1,0],
      [1,1]
  ]).to(device)

  outputs = model(input)

print(outputs)
print(outputs>=0.5)

cuda


<ipython-input-16-ca8695b51878>:16: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
<ipython-input-16-ca8695b51878>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.692
Epoch : 2000, Cost : 0.692
Epoch : 3000, Cost : 0.677
Epoch : 4000, Cost : 0.520
Epoch : 5000, Cost : 0.413
Epoch : 6000, Cost : 0.370
Epoch : 7000, Cost : 0.118
Epoch : 8000, Cost : 0.045
Epoch : 9000, Cost : 0.027
Epoch : 10000, Cost : 0.019
tensor([[0.0122],
        [0.9835],
        [0.9828],
        [0.0312]], device='cuda:0')
tensor([[False],
        [ True],
        [ True],
        [False]], device='cuda:0')


In [ ]:
!pip install koreanize-matplotlib
import numpy as np
import koreanize_matplotlib
import matplotlib.pyplot as plt

In [ ]:
def step(x):
  return np.array(x >0, dtype=np.int)

x = np.arange(-5.0,5.0,0.1)
y = step(x)
plt.title('Step Function')
plt.plot(x,y)
plt.show()

In [ ]:
# 시그모이드 함수
def sigmoid(x):
  return 1/(1+np.exp(-x))

x = np.arange(-5.0, 5.0, 0.1)
y = sigmoid(x)
plt.plot(x,y)
plt.title('Sigmoid Function')
plt.show()

In [ ]:
x = np.arange(-5.0,5.0,0.1)
y = np.tanh(x)
plt.plot(x,y)
plt.grid()
plt.title('Tanh function')
plt.show()

In [ ]:
# ReLU
def relu(x):
  return np.maximum(0,x)

x = np.arange(-5.0, 5.0, 0.1)
plt.plot(x, relu(x))
plt.grid()
plt.title('ReLU Function')
plt.show()

In [ ]:
a = 0.1

def leaky_relu(x):
  return np.maximum(a*x,x)

plt.plot(x,leaky_relu(x))
plt.grid()
plt.title('Leaky ReLU')
plt.show()

In [ ]:
# Softmax
x = np.arange(-5.0,5.0,0.1)
y = np.exp(x)/np.sum(np.exp(x))

plt.plot(x,y)
plt.title('Softmax Function')
plt.show()

# K-Nearest Neighbors

새로운 데이터(x)와 가장 가까운 k개의 데이터를 통해 x를 분류하는 방법   

- euclidean distance
$$\sqrt{(x_1-μ_1)^2 + (x_2-μ_2)^2+ ⋯ + (x_p-μ_p)^2}$$

- manhattan distance
$$|x_1-μ_1|+|x_2-μ_2|+⋯+|x_p-μ_p|$$

      변수의 값이 가지는 스케일의 차이가 모델 학습에 영향을 미치는 것을 막기 위해 스케일링 수행이 필요
      각 변수의 값 차이가 가졌던 정보는 남아있돌록 스케일링


k=1 일때가 가장 이상적이며, 좋은 성능을 보인다
이때 데이터는 노이즈가 없이 어느정도 수준 이상의 처리가 되어 있어야한다.



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import sklearn as sc

In [ ]:
mobile_price = pd.read_csv('train.csv')
print(mobile_price.shape)
mobile_price.head()


In [ ]:
mobile_price.columns

In [ ]:
# 전처리 과정(standardscaler)

columns = mobile_price.columns

X = mobile_price[columns[:-1]]
y = mobile_price[columns[-1]]

SC = StandardScaler()
X = SC.fit_transform(X)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=0
    )

len(x_test[0])



In [ ]:
print(len(x_train), len(x_test))

In [ ]:
for i in range(1,12):

  knn_model=KNeighborsClassifier(n_neighbors=i,
                                metric='manhattan'
                                ).fit(x_train, y_train)

  print(knn_model.score(x_train, y_train))
  print(knn_model.score(x_test, y_test))
  print( '=='*10,'i end')

In [ ]:
knn_model=KNeighborsClassifier(n_neighbors=8,
                                metric='manhattan'
                                ).fit(x_train, y_train)

In [ ]:
knn_model.predict_proba(x_test)[0]

In [ ]:
knn_model.predict(x_test)[0]

In [ ]:
## glass 데이터 실습

glass = pd.read_csv('glass.csv')
print(glass.shape)
glass.head()

In [ ]:
glass.Type.unique()

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x1 = df.iloc[:, 0].values
    self.x2 = df.iloc[:,1].values
    self.x3 = df.iloc[:, 2].values
    self.x4 = df.iloc[:,3].values
    self.x5 = df.iloc[:, 4].values
    self.x6 = df.iloc[:,5].values
    self.x7 = df.iloc[:, 6].values
    self.x8 = df.iloc[:,7].values
    self.x9 = df.iloc[:,8].values
    self.y = df.iloc[:,9].values
    self.length = len(df)

  def __getitem__(self, index):
    x = torch.FloatTensor([self.x1[index], self.x2[index],self.x3[index], 
                           self.x4[index],self.x5[index], self.x6[index],
                           self.x7[index], self.x8[index],self.x9[index]])
    y = torch.LongTensor(self.y-1)[index]
    return x,y

  def __len__(self):
    return self.length


class CustomModel(nn.Module):
  def __init__(self):
    super(CustomModel, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Linear(9,256),
        nn.ReLU()
    )
    self.layer2 = nn.Sequential(
        nn.Linear(256,512),
        nn.ReLU()
    )
    self.layer3 = nn.Sequential(
        nn.Linear(512,128),
        nn.ReLU()
    )
    self.layer4 = nn.Sequential(
        nn.Linear(128,18),
        nn.ReLU()
    )
    self.output_layer = nn.Sequential(
        nn.Linear(18,7),
        nn.Softmax()
    )

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    return x

train_dataset = CustomDataset('glass.csv')
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = 'cuda'
print(device)

model = CustomModel().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(100000):
  cost = 0.0
  for x,y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost += loss

  cost = cost/len(train_dataloader)

  if (epoch + 1) % 1000 ==0:
    print(f'Epoch : {epoch+1:4d}, Cost : {cost:.3f}')

# with torch.no_grad():
#   model.eval()
#   print(glass[:10,:-1])
#   input = torch.FloatTensor([
#       [0,0],
#       [0,1],
#       [1,0],
#       [1,1]
#   ]).to(device)

#   outputs = model(input)

# print(outputs)
# print(outputs<=0.5)

In [17]:
## dataset 자료 살펴보기

pd.read_csv('dataset.csv')['class'].unique()

array(['obtuse triangle', 'acute triangle', 'right triangle'],
      dtype=object)

In [21]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.a = df.iloc[:, 0].values
    self.b = df.iloc[:,1].values
    self.c = df.iloc[:, 2].values
    self.y = df.iloc[:, 3].values
    self.y = list(map(self.string_to_vector, self.y))
    self.length = len(df)

  def string_to_vector(self, value):
    key_value = {'obtuse triangle':2, 'acute triangle':1, 'right triangle':0}
    return key_value[value]

  def __getitem__(self, index):
    x = torch.FloatTensor(sorted([self.a[index], self.b[index], self.c[index]]))
    y = torch.LongTensor(self.y)[index]
    return x,y

  def __len__(self):
    return self.length

class CustomModel(nn.Module):
  def __init__(self):
    super(CustomModel, self).__init__()

    self.layer = nn.Sequential(
        nn.Linear(3, 3)
    )
  def forward(self,x):
    return self.layer(x)

train_dataset = CustomDataset('dataset.csv')
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = 'cpu'

model = CustomModel().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10000):
  cost = 0.0
  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost += loss

  cost = cost/len(train_dataloader)

  if (epoch + 1) % 1000 ==0:
    print(f'Epoch : {epoch+1:4d}, Cost : {cost:.3f}')


Epoch : 1000, Cost : 0.212
Epoch : 2000, Cost : 0.134
Epoch : 3000, Cost : 0.097
Epoch : 4000, Cost : 0.077
Epoch : 5000, Cost : 0.064
Epoch : 6000, Cost : 0.054
Epoch : 7000, Cost : 0.047
Epoch : 8000, Cost : 0.042
Epoch : 9000, Cost : 0.038
Epoch : 10000, Cost : 0.034


In [22]:
# 미분하지않겠다. 평가하겠다.
with torch.no_grad():
  model.eval()
  classes = {2:'obtuse triangle', 1:'acute triangle', 0:'right triangle'}
  inputs = torch.FloatTensor(
      [
          [3.0,4.0,5.0],
       [3,3,3]
      ]
  ).to(device)

In [23]:
model(inputs)

tensor([[ 1.1720, -1.7353, -1.9636],
        [ 1.4544,  3.0996, -4.7665]], grad_fn=<AddmmBackward0>)

In [ ]:
outputs = model(inputs)

In [ ]:
import torch.nn.functional as F
print(torch.round(F.softmax(outputs, dim=1), decimals=2))
print(outputs.argmax(1))
print(list(map(classes.get, outputs.argmax(1).tolist())))